In [145]:
import os
import sys
import time
import pickle
import random
import numpy as np
from tqdm import tqdm_notebook as tqdm

In [146]:
class_num = 10
image_size = 32
img_channels = 3

In [147]:
# ========================================================== #
# ├─ prepare_data()
#  ├─ download training data if not exist by download_data()
#  ├─ load data by load_data()
#  └─ shuffe and return data
# ========================================================== #


In [148]:
def download_data():
    dirname = 'cifar-10-batches-py'
    origin = 'http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz'
    fname = 'cifar-10-python.tar.gz'
    fpath = './' + dirname

    download = False
    if os.path.exists(fpath) or os.path.isfile(fname):
        download = False
        print("DataSet aready exist!")
    else:
        download = True
    if download:
        print('Downloading data from', origin)
        import urllib.request
        import tarfile

        def reporthook(count, block_size, total_size):
            global start_time
            if count == 0:
                start_time = time.time()
                return
            duration = time.time() - start_time
            progress_size = int(count * block_size)
            speed = int(progress_size / (1024 * duration))
            percent = min(int(count * block_size * 100 / total_size), 100)
            sys.stdout.write("\r...%d%%, %d MB, %d KB/s, %d seconds passed" %
                             (percent, progress_size / (1024 * 1024), speed, duration))
            sys.stdout.flush()

        urllib.request.urlretrieve(origin, fname, reporthook)
        print('Download finished. Start extract!', origin)
        if (fname.endswith("tar.gz")):
            tar = tarfile.open(fname, "r:gz")
            tar.extractall()
            tar.close()
        elif (fname.endswith("tar")):
            tar = tarfile.open(fname, "r:")
            tar.extractall()
            tar.close()


In [149]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [150]:
def load_data_one(file):
    batch = unpickle(file)
    data = batch[b'data']
    labels = batch[b'labels']
    print("Loading %s : %d." % (file, len(data)))
    return data, labels

In [151]:
def load_data(files, data_dir, label_count):
    global image_size, img_channels
    data, labels = load_data_one(data_dir + '/' + files[0])
    for f in files[1:]:
        data_n, labels_n = load_data_one(data_dir + '/' + f)
        data = np.append(data, data_n, axis=0)
        labels = np.append(labels, labels_n, axis=0)
    labels = np.array([[float(i == label) for i in range(label_count)] for label in labels])
    data = data.reshape([-1, img_channels, image_size, image_size])
    data = data.transpose([0, 2, 3, 1])
    return data, labels

In [152]:
def prepare_data():
    print("======Loading data======")
    download_data()
    data_dir = './cifar-10-batches-py'
    image_dim = image_size * image_size * img_channels
    meta = unpickle(data_dir + '/batches.meta')

    label_names = meta[b'label_names']
    label_count = len(label_names)
    train_files = ['data_batch_%d' % d for d in range(1, 6)]
    train_data, train_labels = load_data(train_files, data_dir, label_count)
    test_data, test_labels = load_data(['test_batch'], data_dir, label_count)

    print("Train data:", np.shape(train_data), np.shape(train_labels))
    print("Test data :", np.shape(test_data), np.shape(test_labels))
    print("======Load finished======")

    print("======Shuffling data======")
    indices = np.random.permutation(len(train_data))
    train_data = train_data[indices]
    train_labels = train_labels[indices]
    print("======Prepare Finished======")

    return train_data, train_labels, test_data, test_labels


In [153]:
# ========================================================== #
# ├─ _random_crop()
# ├─ _random_flip_leftright()
# ├─ data_augmentation()
# └─ color_preprocessing()
# ========================================================== #

In [154]:
def _random_crop(batch, crop_shape, padding=None):
    oshape = np.shape(batch[0])

    if padding:
        oshape = (oshape[0] + 2 * padding, oshape[1] + 2 * padding)
    new_batch = []
    npad = ((padding, padding), (padding, padding), (0, 0))
    for i in range(len(batch)):
        new_batch.append(batch[i])
        if padding:
            new_batch[i] = np.lib.pad(batch[i], pad_width=npad,
                                      mode='constant', constant_values=0)
        nh = random.randint(0, oshape[0] - crop_shape[0])
        nw = random.randint(0, oshape[1] - crop_shape[1])
        new_batch[i] = new_batch[i][nh:nh + crop_shape[0],
                       nw:nw + crop_shape[1]]
    return new_batch


In [155]:
def _random_flip_leftright(batch):
    for i in range(len(batch)):
        if bool(random.getrandbits(1)):
            batch[i] = np.fliplr(batch[i])
    return batch

In [156]:
def color_preprocessing(x_train, x_test):
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train[:, :, :, 0] = (x_train[:, :, :, 0] - np.mean(x_train[:, :, :, 0])) / np.std(x_train[:, :, :, 0])
    x_train[:, :, :, 1] = (x_train[:, :, :, 1] - np.mean(x_train[:, :, :, 1])) / np.std(x_train[:, :, :, 1])
    x_train[:, :, :, 2] = (x_train[:, :, :, 2] - np.mean(x_train[:, :, :, 2])) / np.std(x_train[:, :, :, 2])

    x_test[:, :, :, 0] = (x_test[:, :, :, 0] - np.mean(x_test[:, :, :, 0])) / np.std(x_test[:, :, :, 0])
    x_test[:, :, :, 1] = (x_test[:, :, :, 1] - np.mean(x_test[:, :, :, 1])) / np.std(x_test[:, :, :, 1])
    x_test[:, :, :, 2] = (x_test[:, :, :, 2] - np.mean(x_test[:, :, :, 2])) / np.std(x_test[:, :, :, 2])

    return x_train, x_test

In [157]:
def data_augmentation(batch):
    batch = _random_flip_leftright(batch)
    batch = _random_crop(batch, [32, 32], 4)
    return batch


In [158]:
import tensorflow as tf
#from tflearn.layers.conv import global_avg_pool
from tensorflow.contrib.layers import batch_norm, flatten
from tensorflow.contrib.layers import xavier_initializer
from tensorflow.contrib.framework import arg_scope
tf.reset_default_graph()
#from cifar10 import *


In [159]:
# Hyperparameter
growth_k = 24
nb_block = 2 # how many (dense block + Transition Layer) ?
#init_learning_rate = 1e-4 # AdamOptimizer epsilon

epsilon = 1e-4 # AdamOptimizer epsilon
dropout_rate = 0.2

# Momentum Optimizer will use
nesterov_momentum = 0.9
weight_decay = 1e-4
init_learning_rate = 0.1

# Label & batch_size
batch_size = 64

iteration = 782
# batch_size * iteration = data_set_number

test_iteration = 10

total_epochs = 300

In [160]:
def Evaluate(sess):
    test_acc = 0.0
    test_loss = 0.0
    
    
    for it in range(test_iteration):

        test_feed_dict = {
            handle: test_handle,
            learning_rate: epoch_learning_rate,
            training_flag: False
        }
        
        try:
            loss_, acc_ = sess.run([cost, accuracy], feed_dict=test_feed_dict)
        
        except tf.errors.OutOfRangeError:
            print("End of append test.")

        test_loss += loss_ / 10.0
        test_acc += acc_ / 10.0

    summary = tf.Summary(value=[tf.Summary.Value(tag='test_loss', simple_value=test_loss),
                                tf.Summary.Value(tag='test_accuracy', simple_value=test_acc)])

    return test_acc, test_loss, summary

In [161]:
def conv_layer(input, filter, kernel, stride=1, layer_name="conv"):
    with tf.name_scope(layer_name):
        network = tf.layers.conv2d(inputs=input, use_bias=True, filters=filter, kernel_size=kernel, strides=stride, padding='SAME')
        return network
    
def global_avg_pool(x, name='global_avg_pool', data_format='NHWC'):
    assert x.shape.ndims == 4
    assert data_format in ['NHWC', 'NCHW']
    with tf.name_scope(name):
        axis = [1, 2] if data_format == 'NHWC' else [2, 3]
        return tf.reduce_mean(x, axis)

def Global_Average_Pooling(x, stride=1):

    return global_avg_pool(x, name='Global_avg_pooling')

def Drop_out(x, rate, training) :
    return tf.layers.dropout(inputs=x, rate=rate, training=training)

def Relu(x):
    return tf.nn.relu(x)

def Average_pooling(x, pool_size=[2,2], stride=2, padding='VALID'):
    return tf.layers.average_pooling2d(inputs=x, pool_size=pool_size, strides=stride, padding=padding)


def Max_Pooling(x, pool_size=[3,3], stride=2, padding='VALID'):
    return tf.layers.max_pooling2d(inputs=x, pool_size=pool_size, strides=stride, padding=padding)

def Concatenation(layers) :
    return tf.concat(layers, axis=3)

def Linear(x) :
    return tf.layers.dense(inputs=x, units=class_num, name='linear')


In [162]:
class DenseNet():
    def __init__(self, x, nb_blocks, filters, training):
        self.nb_blocks = nb_blocks
        self.filters = filters
        self.training = training
        self.model = self.Dense_net(x)


    def bottleneck_layer(self, x, scope):
        # print(x)
        with tf.name_scope(scope):
            #x = Batch_Normalization(x, training=self.training, scope=scope+'_batch1')
            #x = tf.layers.batch_normalization(x, training=self.training)
            x = tf.contrib.layers.batch_norm(x, scale=True, is_training=self.training, updates_collections=None)
            x = Relu(x)
            x = conv_layer(x, filter=4 * self.filters, kernel=[1,1], layer_name=scope+'_conv1')
            x = Drop_out(x, rate=dropout_rate, training=self.training)

            #x = Batch_Normalization(x, training=self.training, scope=scope+'_batch2')
            #x = tf.layers.batch_normalization(x, training=self.training)
            x = tf.contrib.layers.batch_norm(x, scale=True, is_training=self.training, updates_collections=None)
            x = Relu(x)
            x = conv_layer(x, filter=self.filters, kernel=[3,3], layer_name=scope+'_conv2')
            x = Drop_out(x, rate=dropout_rate, training=self.training)

            # print(x)

            return x

    def transition_layer(self, x, scope):
        with tf.name_scope(scope):
            #x = Batch_Normalization(x, training=self.training, scope=scope+'_batch1')
            #x = tf.layers.batch_normalization(x, training=self.training)
            x = tf.contrib.layers.batch_norm(x, scale=True, is_training=self.training, updates_collections=None)
            x = Relu(x)
            shape = x.get_shape().as_list()
            in_channel = shape[3]
            #x = conv_layer(x, filter=self.filters, kernel=[1,1], layer_name=scope+'_conv1')
            x = conv_layer(x, filter=in_channel*0.5, kernel=[1,1], layer_name=scope+'_conv1')
            x = Drop_out(x, rate=dropout_rate, training=self.training)
            x = Average_pooling(x, pool_size=[2,2], stride=2)

            return x

        
    def dense_block(self, input_x, nb_layers, layer_name):
        with tf.name_scope(layer_name):
            layers_concat = list()
            layers_concat.append(input_x)
            #print(input_x.shape)
            x = self.bottleneck_layer(input_x, scope=layer_name + '_bottleN_' + str(0))
            #print("after:",input_x.shape)
            layers_concat.append(x)

            for i in range(nb_layers - 1):
                x = Concatenation(layers_concat)
                x = self.bottleneck_layer(x, scope=layer_name + '_bottleN_' + str(i + 1))
                layers_concat.append(x)

            x = Concatenation(layers_concat)

            return x

    def Dense_net(self, input_x):
        x = conv_layer(input_x, filter=16, kernel=[3,3], stride=1, layer_name='conv0')
        #x = Max_Pooling(x, pool_size=[3,3], stride=2)

        x = self.dense_block(input_x=x, nb_layers=3, layer_name='dense_1')
        x = self.transition_layer(x, scope='trans_1')

        x = self.dense_block(input_x=x, nb_layers=3, layer_name='dense_2')
        x = self.transition_layer(x, scope='trans_2')
        
        x = self.dense_block(input_x=x, nb_layers=3, layer_name='dense_final')



        # 100 Layer
        #x = Batch_Normalization(x, training=self.training, scope='linear_batch')
        #x = tf.layers.batch_normalization(x, training=self.training)
        x = tf.contrib.layers.batch_norm(x, scale=True, is_training=self.training, updates_collections=None)
        x = Relu(x)
        x = Global_Average_Pooling(x)
        x = flatten(x)
        x = Linear(x)


        # x = tf.reshape(x, [-1, 10])
        return x


In [163]:
#tf.reset_default_graph()
train_x, train_y, test_x, test_y = prepare_data()
train_x, test_x = color_preprocessing(train_x, test_x)


train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
train_dataset = train_dataset.batch(64)
train_dataset = train_dataset.prefetch(64)
train_dataset = train_dataset.repeat(300)
test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y))
test_dataset = test_dataset.batch(1000)
test_dataset = test_dataset.prefetch(1000)
#extra_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
#extra_dataset = extra_dataset.batch(64)

handle = tf.placeholder(tf.string, [])
feed_iterator = tf.data.Iterator.from_string_handle(handle, train_dataset.output_types,
                                                      train_dataset.output_shapes)
images, labels = feed_iterator.get_next()

train_iterator = train_dataset.make_one_shot_iterator()
test_iterator = test_dataset.make_initializable_iterator()




# image_size = 32, img_channels = 3, class_num = 10 in cifar10

training_flag = tf.placeholder(tf.bool)
learning_rate = tf.placeholder(tf.float32, name='learning_rate')

logits = DenseNet(x=images, nb_blocks=nb_block, filters=growth_k, training=training_flag).model
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits))

l2_loss = tf.add_n([tf.nn.l2_loss(var) for var in tf.trainable_variables()])
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=nesterov_momentum, use_nesterov=True)
train = optimizer.minimize(cost + l2_loss * weight_decay)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#saver = tf.train.Saver(tf.global_variables())
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

======Loading data======
DataSet aready exist!
Loading ./cifar-10-batches-py/data_batch_1 : 10000.
Loading ./cifar-10-batches-py/data_batch_2 : 10000.
Loading ./cifar-10-batches-py/data_batch_3 : 10000.
Loading ./cifar-10-batches-py/data_batch_4 : 10000.
Loading ./cifar-10-batches-py/data_batch_5 : 10000.
Loading ./cifar-10-batches-py/test_batch : 10000.
Train data: (50000, 32, 32, 3) (50000, 10)
Test data : (10000, 32, 32, 3) (10000, 10)
======Load finished======
======Shuffling data======
======Prepare Finished======


In [164]:
total = np.sum([np.prod(v.get_shape().as_list()) for v in tf.trainable_variables()])
print(total)

258420


In [165]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True # prevents allocation of all GPU memory

with tf.Session(config=config) as sess:
#with tf.Session() as sess:
    
    #ckpt = tf.train.get_checkpoint_state('./model')
    #if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
        #saver.restore(sess, ckpt.model_checkpoint_path)
    #else:
    sess.run(init_op)
    train_handle = sess.run(train_iterator.string_handle())
    test_handle = sess.run(test_iterator.string_handle())

    summary_writer = tf.summary.FileWriter('./logs5', sess.graph)

    epoch_learning_rate = init_learning_rate
        
    for epoch in range(1, total_epochs + 1):
        if epoch == (total_epochs * 0.5) or epoch == (total_epochs * 0.75):
            epoch_learning_rate = epoch_learning_rate / 10

        
        train_acc = 0.0
        train_loss = 0.0

        for step in tqdm(range(1, iteration + 1)):
            

            train_feed_dict = {
                handle: train_handle,
                learning_rate: epoch_learning_rate,
                training_flag : True
            }
            
            try:
                _, batch_loss, batch_acc= sess.run([train, cost, accuracy], feed_dict=train_feed_dict)
                #batch_acc = accuracy.eval(feed_dict=train_feed_dict)
                
            except tf.errors.OutOfRangeError:
                print("End of append train.",step)

            train_loss += batch_loss
            train_acc += batch_acc


            if step == iteration :
                
                sess.run(test_iterator.initializer)
                
                train_loss /= iteration # average loss
                train_acc /= iteration # average accuracy

                train_summary = tf.Summary(value=[tf.Summary.Value(tag='train_loss', simple_value=train_loss),
                                                  tf.Summary.Value(tag='train_accuracy', simple_value=train_acc)])

                test_acc, test_loss, test_summary= Evaluate(sess)

                summary_writer.add_summary(summary=train_summary, global_step=epoch)
                summary_writer.add_summary(summary=test_summary, global_step=epoch)
                summary_writer.flush()

                line = "epoch: %d/%d, train_loss: %.4f, train_acc: %.4f, test_loss: %.4f, test_acc: %.4f \n" % (
                    epoch, total_epochs, train_loss, train_acc, test_loss, test_acc)
                print(line)

                with open('logs5.txt', 'a') as f :
                    f.write(line)



        #saver.save(sess=sess, save_path='./model/dense.ckpt')


epoch: 1/300, train_loss: 1.3842, train_acc: 0.4948, test_loss: 2.7945, test_acc: 0.2820 



epoch: 2/300, train_loss: 0.9860, train_acc: 0.6466, test_loss: 1.5853, test_acc: 0.4731 



epoch: 3/300, train_loss: 0.8638, train_acc: 0.6911, test_loss: 1.6244, test_acc: 0.4634 



epoch: 4/300, train_loss: 0.7842, train_acc: 0.7227, test_loss: 1.5362, test_acc: 0.5334 



epoch: 5/300, train_loss: 0.7261, train_acc: 0.7435, test_loss: 1.0918, test_acc: 0.6226 



...

epoch: 32/300, train_loss: 0.4319, train_acc: 0.8495, test_loss: 2.0550, test_acc: 0.5236 



epoch: 33/300, train_loss: 0.4292, train_acc: 0.8493, test_loss: 0.6600, test_acc: 0.7826 



epoch: 34/300, train_loss: 0.4254, train_acc: 0.8518, test_loss: 0.6819, test_acc: 0.7803 



epoch: 35/300, train_loss: 0.4255, train_acc: 0.8509, test_loss: 0.8433, test_acc: 0.7369 



KeyboardInterrupt: 

In [228]:
#progress bar - tqdm
#count of params 
#tf.data 